# From Statistical Models to LLMs
## Learning Goals
- Understand how statistical n-gram models work.
- Compare their limitations with modern LLMs.
- See how context window size impacts predictions.
- Experience emergent behaviors in transformer-based models.

This notebook connects with Section *1.2 From Statistical Language Models to Neural-based LLMs* of the lecture notes.

## Step 1: Load a sample text corpus
We’ll use the **Reuters dataset** from NLTK, which contains short news articles.  
This will serve as training data for our n-gram models.

In [9]:
import nltk
from nltk.util import ngrams
from collections import Counter, defaultdict
import random

nltk.download('reuters')
nltk.download('punkt')
nltk.download('punkt_tab')

from nltk.corpus import reuters
sentences = reuters.sents(categories='trade')
tokens = [t.lower() for sent in sentences for t in sent]
print('Number of tokens:', len(tokens))
print('Sample tokens:', tokens[:30])

[nltk_data] Downloading package reuters to /home/ebezerra/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to /home/ebezerra/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ebezerra/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Number of tokens: 142731
Sample tokens: ['asian', 'exporters', 'fear', 'damage', 'from', 'u', '.', 's', '.-', 'japan', 'rift', 'mounting', 'trade', 'friction', 'between', 'the', 'u', '.', 's', '.', 'and', 'japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'asia', "'"]


## Step 2: Build a simple n-gram model
- A **bigram** model looks at the last 1 token.
- A **trigram** model looks at the last 2 tokens.

We’ll count frequencies and use them to predict the next word.

In [10]:
def build_ngram_model(tokens, n=2):
    model = defaultdict(Counter)
    for i in range(len(tokens)-n):
        context = tuple(tokens[i:i+n-1])
        next_word = tokens[i+n-1]
        model[context][next_word] += 1
    return model

bigram_model = build_ngram_model(tokens, n=2)
trigram_model = build_ngram_model(tokens, n=3)

def generate_text(model, n=2, length=20, seed=None):
    if not seed:
        seed = random.choice(list(model.keys()))
    output = list(seed)
    for _ in range(length):
        context = tuple(output[-(n-1):])
        if context not in model:
            break
        next_word = model[context].most_common(1)[0][0]
        output.append(next_word)
    return ' '.join(output)

print('Bigram sample:', generate_text(bigram_model, n=2))
print('Trigram sample:', generate_text(trigram_model, n=3))

Bigram sample: drafting amendments to the u . s . s . s . s . s . s . s . s
Trigram sample: last this century and the united states , the central bank said . " the u . s . trade deficit ,


### Reflection
- Does the output feel natural or broken?
- Notice how **short context** (bigram/trigram) limits coherence.
- Try changing the `length` parameter and observe when it becomes gibberish.

## Step 3: Use a Transformer-based LLM (GPT-2)
Now let’s compare with a pretrained HuggingFace model.  
Unlike n-grams, GPT-2 has a context window of **1024 tokens** and was trained on a massive dataset.

In [11]:
from transformers import pipeline
generator = pipeline('text-generation', model='gpt2', device=-1)  # device=-1 = CPU

prompt = 'The future of artificial intelligence in databases'
output = generator(prompt, max_length=50, num_return_sequences=1)
print(output[0]['generated_text'])

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The future of artificial intelligence in databases is uncertain. As some of the companies that are working on AI with the help of IBM come up with new ways to manage and process data, such as the DeepMind AI project, it is possible that there will be a shift to a more collaborative approach to AI.

Answers to Questions


In [14]:
from rich.console import Console
from rich.markdown import Markdown

console = Console()
text = output[0]['generated_text']

console.rule("[bold blue] Generated Text [/bold blue]")
console.print(Markdown(text))


────────────────────────────────────────────────  Generated Text  ─────────────────────────────────────────────────

The future of artificial intelligence in databases is uncertain. As some of the companies that are working on AI   
with the help of IBM come up with new ways to manage and process data, such as the DeepMind AI project, it is      
possible that there will be a shift to a more collaborative approach to AI.                                        

Answers to Questions

### Reflection
- The GPT-2 output is more coherent, even though the model is relatively small.
- Unlike n-grams, it can capture longer dependencies.
- This illustrates the **scaling → emergent capabilities** phenomenon.

## Exercises
1. Train a 4-gram model and compare with the bigram/trigram. Does it improve coherence?
2. Replace `gpt2` with a larger HuggingFace model (e.g., `distilgpt2`, `gpt2-medium`) and compare.
3. Change the `prompt` and observe how the model continues your text.